# Dealing with Generalisation

- Abstraction is a common design pattern, and has its own set of refactoring techniques
- This is mainly to move functionality along the class inheritance hierarchy, creating new classes and interfaces, and replacing inheritance with delegation and vice versa

## 1. Pull Up Field


- Problem
    - Two classes/subclasses have the same field

- Solution
    - Put field into parent class

- Motivation
    - Avoid repetition

- Relationships with other refactoring methods
    - Opposite 
        - Push down field
    - Similar
        - Pull up method

- Related code smells
    - Duplicate Code

- Example:

In [1]:
class SuperClass:
    def __init__(self):
        ## add attr here instead
        self.attr = 123

class SubClass1(SuperClass):
    def __init__(self):
        ## Remove attr from here
        # self.attr = 123
        ...

class SubClass2(SuperClass):
    def __init__(self):
        ## Remove attr from here
        # self.attr = 123
        ...

## 2. Pull Up Method

- Problem
    - You have subclasses with methods that perform the same/similar jobs

- Solution
    - Pull the method into the superclass

- Motivation
    - Minimise repetition

- Relationships with other refactoring methods
    - Opposite 
        - Push Down Method
    - Similar
        - Pull Up Field
    - Related
        - Form Template Method

- Related code smells
    - Duplicate Code

- Example:

## 3. Pull Up Constructor Body

- Problem
    - You have subclasses with constructors that are almost identical (i.e. same attributes provided etc)

- Solution
    - Pull the attribute definition into the superclass

- Motivation
    - Minimise repetition

- Relationships with other refactoring methods
    - Similar
        - Pull Up Method

- Related code smells
    - Duplicate Code

- Example:

In [ ]:
class Employee:
    def __init__(self, name, id, gender):
        ## Put name, gender, ID here, instead of in the subclasses
        self.name=name
        self.id=id
        self.gender=gender

class Manager(Employee):
    def __init__(self, name, id, gender):
        ## Call superclass init
        super().__init__(name, id, gender)

class Minion(Employee):
    def __init__(self, name, id, gender):
        ## Call superclass init
        super().__init__(name, id, gender)

## 4. Push Down Method

- Problem
    - You have behaviour in the superclass that is used by only 1 or very few subclasses

- Solution
    - Move behaviour to subclass

- Motivation
    - More coherence in the superclass, place methods where they are used

- Relationships with other refactoring methods
    - Opposite 
        - Pull Up Method
    - Similar
        - Push Down Field
    - Related
        - Extract Subclass

- Related code smells
    - Refused Bequest

- Example:

## 5. Push Down Field

- Problem
    - You have a field in the superclass that is only used in the subclass

- Solution
    - Push the field into the subclass

- Motivation
    - Minimise fields in superclass that are unnecessary
    - Better class coherence

- Relationships with other refactoring methods
    - Opposite 
        - Pull up field
    - Similar
        - Push down method
    - Related
        - Extract subclass

- Related code smells
    - Refused Bequest

- Example:

## 6. Extract Subclass

- Problem
    - You have have code in your superclass that returns a different behaviour depending on the signature of the input

- Solution
    - Pull out special behaviour into a subclass, and use polymorphism

- Motivation
    - Less if/else makes for cleaner flow, and easier refactoring

- Relationships with other refactoring methods
    - Similar
        - Extract class

- Related code smells
    - Large Class

- Example:

In [ ]:
class bad__Employee:
    def __init__(self, type: str):
        self.type=type

    def get_pay(self):
        if self.type == 'manager':
            return 100
        else:
            return 80
        
class good__Employee:
    def get_pay(self) -> float:
        return 80
    
class Manager(good__Employee):
    def get_pay(self) -> float:
        return 100
    

## 7. Extract Superclass

- Problem
    - You have two classes with common fields/methods

- Solution
    - Make a shared superclass and moved the repeated stuff to the superclass

- Motivation
    - Less repetition of code. 
    - Single source of truth, when things need to change, you only need to edit 1 class instead of 2

- Relationships with other refactoring methods
    - Similar
        - Extract Interface

- Related code smells
    - Duplicate Code

- Example:

In [1]:
class Animal:
    def __init__(self, legs):
        self.legs=legs

    def count_legs(self):
        return self.legs
    
class Person(Animal):
    ## Move `legs` field to superclass
    def __init__(self):
        super().__init__(legs=2)
        # self.legs = 2

class Dog(Animal):
    ## Move `legs` field to superclass
    def __init__(self):
        super().__init__(legs=2)
        # self.legs = 4


## 8. Extract Interface


- Problem
    - You have multiple classes implementing similar pattern of methods
    - AND these classes don't have a clear group for you to group them together as a superclass

- Solution
    - Rather than rely on inheritance, define an interface, and let your classes implement this interface
    - In Python, you can't really define an interface. As of 2024, you have 2 options; either define a `typing.Protocol` class and let your static type checkers handle duck-typing, or inherit from the good old `abc.ABC` with `@abstratmethod`

- Motivation
    - There are occasions where you have some extensible library of objects, and you want to ensure that anything new MUST implement some minimum set of methods
    - Making an explicit interface and requiring all new additions to implement it helps coordinate code
    - This is almost like `Extract Superclass` that we saw previously, except that a superclass can contain actual implementations of methods and attributes, not just a common interface

- Relationships with other refactoring methods
    - Similar
        - Extract Superclass

- Example:

In [19]:
from abc import ABC, abstractmethod
from typing import Protocol, runtime_checkable

@runtime_checkable
class HasLegsProtocol(Protocol):
    legs: int

    def count_legs(self) -> int:
        ...
    
class HasLegsBaseClass(ABC):
    def __init__(self, legs: int):
        self.legs=legs

    @abstractmethod
    def count_legs(self) -> int:
        ...
    
class PersonProtocolImplicit:
    def __init__(self):
        self.legs: int =2

    def count_legs(self) -> int:
        return self.legs
    
class PersonProtocolExplicit(HasLegsProtocol):
    def __init__(self):
        self.legs: int =2

    def count_legs(self) -> int:
        return self.legs

class PersonBaseClass(HasLegsBaseClass):
    def __init__(self):
        self.legs: int =2

    def count_legs(self) -> int:
        return self.legs


test1 = PersonProtocolImplicit()
print(isinstance(test1, HasLegsProtocol)) ##Protocol signature is implemented, so this returns True even without explicit inheritance

test2 = PersonProtocolExplicit()
print(isinstance(test2, HasLegsProtocol)) ##We can still inherit explicitly from protocol

test3 = PersonBaseClass()
print(isinstance(test3, HasLegsBaseClass)) ##In the same way, we can simply rely on inheritance from an abc
print(isinstance(test3, HasLegsProtocol)) ##BUT even without explicitly stating the inheritance, notice that our object is STILL an instance of the protocol!

True
True
True
True


## 9. Collapse Hierarchy

- Problem
    - You have a class hierachy where a subclass is the same as the superclass

- Solution
    - Remove either subclass or superclass, since there is no actual need for inheritance

- Motivation
    - Don't specify inheritance relationships unless it is needed. Else you have 2 points of failure instead of 1

- Relationships with other refactoring methods
    - Similar
        - Inline Class

- Related code smells
    - Lazy Class
    - Speculative Generality

- Example:

## 10. Form Template Method

- Problem
    - Your subclasses implement a method that have the same steps, but with slight differences in each step

- Solution
    - Put a "template" of the method in the superclass, and delegate the specific implementations of the substeps into the subclasses

- Motivation
    - Minimise repetition
    - As much as possible, let your code have a single source of truth

- Related Design Pattern
    - Template Method

- Related code smells
    - Duplicate Code

- Example:

In [22]:
from abc import ABC, abstractmethod

class TaxableResidence:
    
    ## Implement template for computing tax here...
    def get_tax_amount(self):
        base_amount = self.get_base_amount()
        final_tax_amount = self.adjust_base_amount(base_amount)
        return final_tax_amount
    
    ## ...but delegate the implementation of the substeps to the subclasses
    @abstractmethod
    def get_base_amount(self) -> float:
        ...

    @abstractmethod
    def adjust_base_amount(self, base_amount) -> float:
        ...

class HDBResidence(TaxableResidence):
    ## Implementation of substeps here
    def get_base_amount(self) -> float:
        return 100
    
    def adjust_base_amount(self, base_amount) -> float:
        return base_amount*1.1

class GCBResidence(TaxableResidence):
    def get_base_amount(self) -> float:
        return 200
    
    def adjust_base_amount(self, base_amount) -> float:
        return base_amount*1.2


hdb = HDBResidence()
gcb = GCBResidence()
hdb.get_tax_amount()
gcb.get_tax_amount()

240.0

## 11. Replace Inheritance with Delegation

- Problem
    - You have a subclass that uses only a portion of the superclass's methods

- Solution
    - Rather than inheritance, consider using composition instead 
    - That is, remove the inheritance relationship, and pass an instance of the superclass as a field 
    - Then, delegate all methods to the original superclass

- Motivation
    - Don't have inheritance relationships unless you REALLY want 2 things to be coupled tightly
    - Inheritance is much harder to maintain

- Downside
    - Delegation requires you to write delegation methods, which an lead to a lot of your class's method just being middlemen

- Relationships with other refactoring methods
    - Opposite 
        - Replace Delegation with Inheritance

- Related Design Pattern
    - Strategy

- Example:

In [25]:
class DataProcessor:
    def __init__(self):
        ...
    
    def process_data_1(self):
        print('process_data 1')
        
    def process_data_2(self):
        print('process_data 2')

class bad__Reporter(DataProcessor):
    def __init__(self):
        ...

class good__Reporter():
    def __init__(self):
        self.processor = DataProcessor()

    ## Trading off more lines of code, with no inheritance structure
    def process_data_1(self):
        self.processor.process_data_1()
    
    def process_data_2(self):
        self.processor.process_data_2()

br = bad__Reporter()
br.process_data_1()
br.process_data_2()

gr = good__Reporter()
gr.process_data_1()
gr.process_data_2()

process_data 1
process_data 2
process_data 1
process_data 2


## 12. Replace Delegation with Inheritance

- Problem
    - A class contains many methods that just delegate to another class

- Solution
    - Let the class containing the methods inherit from the class instead of delegating

- Motivation
    - Minimise lines of code
    - This is simply the inverse of `Replace Inheritance with Delegation`
    - As stated, the trade off here is between how many lines of code your class has, balanced against the difficulty of maintaining the inheritance relationship
    
- Relationships with other refactoring methods
    - Opposite 
        - Replace Inheritance with Delegation
    - Similar
        - Remove Middle Man

- Related code smells
    - Inappropriate Intimacy

- Example: